# Prospecção de Dados (Data Mining) DI/FCUL - TP01


## Processing large quantities of data - an exposition for text procesing

#### A simple tutorial by Andre Falcao (MC/DI/FCUL - 2023 - 202)

This tutorial will try to solve a very simple problem: **from a set of documents, can we identify the ones that are more similar to each other?**

The first part of this tutorial will focus on [18 of La Fontaine fables](https://en.wikipedia.org/wiki/La_Fontaine%27s_Fables), translated into English by William Trowbridge and made available to the public by [Project Gutenberg](https://www.gutenberg.org/ebooks/24108). The Text file was editted removing headers and copyright information as they would add noise to the data set. Each fable is then one document and the basic question stands: What documents are more similar to others

First let's open our data set and print it out, as it is rather small (and actually entertaining)

In [ ]:
stories=open("lafontaine.txt", "rt").readlines()

all_word_counts=[]
texts=[]
for i,story in enumerate(stories):
    print("Story %d:\n %s" % (i, story))

Text  retrieved from the web many times contains weird characters that do not add content and end up bycreating artificial differences, so we will use the following 3 utilitary functions

* The simplest possible `tokenizer` (this will eventually be replaced in the future with a more robust solution)
* `remove_accents()` - this will remove all accents from a unicode text replacing it with the equivalent nonaccented words
* `remove_stuff()` - will remove many weird and strange characters that occasionally appear in text scraped from the web

In [ ]:
import unicodedata

def basic_word_tokenizer(text):
    return text.split()

def remove_accents(s):
    nfkd_form = unicodedata.normalize('NFKD', s)
    return u"".join([c for c in nfkd_form if not unicodedata.combining(c)])

def remove_stuff(s):
    for c in "\\\t0123456789Ææœ—‘’\ufeff{|}“”.,()$£%&[]?@#!=;*+–\"ǁ":
        s=s.replace(c, "")
    s=s.replace("-", " ")
    return s


We will now add another function, that will use all these utilities, for extracting all the words form a set of documents (a corpus)


In [ ]:

def get_words_from_corpus(corpus):
    words_texts=[]
    for i,text in enumerate(corpus):
        text=text.strip().lower()
        text=remove_accents(text)
        text=remove_stuff(text)
        text=text.lower()
        words = basic_word_tokenizer(text)
        words_texts.append(words)
    return words_texts


words_texts = get_words_from_corpus(stories)

#let's print out the first 20 words of the first five stories
for i, words in enumerate(words_texts[:5]):
    print("Story %d:\n %s" %(i, words[:20]))

Other things we will eventually need is to have the unique words of each document and also  the set of all the unique words in a given corpus

We will accomplish both objectives with Python sets

In [ ]:
def calc_all_words(words_text_sets):
    all_words=set()
    for words in words_text_sets: all_words |= words
    return all_words

words_text_sets = [set(words) for words in words_texts]
all_words=calc_all_words(words_text_sets)
for i, word in enumerate(list(all_words)[:20]):
    print("Word %d: %s" %(i, word))

### Question: What are the stories that are most similar in content?

### Solution 1:  we might compare by checking how many words each story shares and make a table

We could do this by making sets of the existing words in each text, which will take out all repetitions and then make comparisons among all sets. A very common way of doing that would be to compute the Cosine Similarity that checks the ratio of the scores of the common words of a document 

$$CosSim=\dfrac{\sum_i{A_i.B_i}}{\sqrt{\sum_i{A_i^2}}.\sqrt{\sum_i{B_i^2}}}$$


for a simple approach  A_i and B_i correspond to the presence of word $i$ in Document $A$ or document $B$

Another way to make comparisons would be to use the jaccard score that could be easily computed with python sets as it is just the ratio of the intersection with the union of both sets

$$Jaccard=\dfrac{T_a \cap T_b}{T_a \cup T_b}$$



In [ ]:
words_text_sets=[set(words) for words in words_texts]
from math import sqrt
def cos_similarities(word_sets):
    sims=[]
    for a in range(len(word_sets)-1):
        for b in range(a+1, len(word_sets)):
            CosSim=len(word_sets[a] & word_sets[b]) / (sqrt(len(word_sets[a])*len(word_sets[b])))
            sims.append((CosSim, (a,b)))
    return sims


sims = cos_similarities(words_text_sets)

#now let's sort the list in descending order so that the most similar stories will appear
sims = sorted(sims, reverse=True)

#now let's get the 5 most common text pairs
for J, text_pair in sims[:5]: 
    print( "%s has similarity: %7.4f" % (text_pair, J))

This function willl be critical in assessing the similarity of many pairs of documents thus it will be critical to assess its performance

In [ ]:
%timeit sims = cos_similarities(words_text_sets)

#### Exercises: 
1. Modify the text_similarity so that it will use the Jaccard score 
2. Compare the results and discuss them
3. Check its performance and comment the differences
4. [to do at home!] can the above function be sped up if instead of using strings for words we use integers as indices for each individual word

In [ ]:
# !!!!!!!! solution 1 !!!!!!!!!
def jacc_similarities(word_sets):
    pass

In [ ]:
%timeit sims = jacc_similarities(words_text_sets)

In [ ]:
# !!!!!!!! solution of exercise 4 !!!!!!!!!

# first let's index each word

#finally, let's (re)construct the stories with the word indexes:

# now with our stories properly word indexed we can recompute the 

# similarities which shuld be the same and we will print out  the 5 similar stories

    
#Let's verify the computing time
%timeit sims = cos_similarities(idxs_text_sets)

## A different solution

The solution above is ok but we are not actually entering into  our computation the following issues:
1. **Words have different importances**. The word "the" or "a" very much common words have different importance than "eagle", "running" or "manners"
2. **We are disregarding the amount of times a word appear on a text**. A word more common should count more then a word that appears only once


### Introducing TF.IDF - A way off the quagmire

TFIDF is a calculation that allows to assign values to each word in our corpus and it is a product of 2 values 

$$ TFIDF_{ij} = TF_{ij}.IDF_i $$

### Defining TF

first let's define TF. which solves our Problem 2. How to emphasize the importance that each word has in a text? This score is defined as

$$ TF_{ij} = \dfrac{C_{ij}}{max_k(C_{kj})} $$

where $C_{ij}$ is the number of times term $i$ appears on document $j$. So we will need a `word counter` that should be able to count the number of occurrences of each word on each text

In [ ]:
def word_counter(words):
    #1. get uniques
    unique_words=set(words)
    #2. make a dictionary for counting
    D=dict(zip(unique_words, [0]*len(unique_words)))
    #3. count all
    for w in words: D[w]+=1
    return D  

def TF(word_counts):
    #get the counts
    counts  = word_counts.values()
    if len(counts)==0: return {}
    the_max = max(counts)   #compute the maximum
    #return a dictionary that for each word returns the ratio of wc/max_wc
    return dict(zip(word_counts.keys(), [c/the_max for c in counts]))    

#let's try the above functions for story one
text=words_texts[0]
wcounts =word_counter(text)
tfs = TF(wcounts)



and let's look at the TFs for the first 10 words of this little story:


In [ ]:
for word in text[:10]:
    print("%12s - Counts:%3d; TF: %7.4f" % (word, wcounts[word], tfs[word]))

Notice that the TFs correspond to one text only so we have to compute the TFs for all documents in a new function

The function will return the TFs for each document in a array

In [ ]:
def calc_tfs(words_texts):
    all_tfs=[]
    for words in words_texts:
        wcounts = word_counter(words)
        all_tfs.append(TF(wcounts))
    return all_tfs

all_tfs = calc_tfs(words_texts)

#### Defining IDF

IDF aims at solving problem 1. Not all words are equal, and some have more meaning than others. Words that appear in all documents of a corpus should not be that interesting, while some words quite rare should be scored more heavily.  

$$ IDF_i = log_2{\dfrac{N}{n_i}} $$

Where N is the total number of words in a corpus and $n_i$ is the number of documents in which the word $i$ appears in. This requires us to have a complete list of the words. The IDF will assign a global score to each word. The more uncommon it is, the higher the score. Words that appear always, like "the" or "a" will have score of zero, or very close

In [ ]:
from math import log2

def IDF(all_words, doc_word_counts):
    #first initialize a new dictionary with one entry for each word
    D=dict(zip(all_words, [0]*len(all_words)))
    N=len(doc_word_counts)
    for doc in doc_word_counts:
        for word in doc: D[word]+=1
    return {w: log2(N/D[w]) for w in D}

idfs = IDF(all_words, words_text_sets)


With the global IDFs computed we can now show the results of the first 10 words of the first fable

In [ ]:
text=words_texts[0]
wcounts =word_counter(text)
tfs=TF(wcounts)

for word in text[:10]:
    print("%12s - Counts:%3d; TF: %7.4f IDF: %7.4f -> TFIDF: %7.4f" % (word, wcounts[word], 
                                                                       tfs[word], idfs[word],
                                                                       tfs[word]*idfs[word]))

### Computing document similarity with TFIDF

We will now make a little function that receives the indexes of the documents, the unique words of both documents, and the TFs and IDFs previously computed and will produce the cosine distance for each pair of documents

We are going to use numpy for its broadcasting capabilities that will speed up the calculations of the similarities



In [ ]:
import numpy as np
def cosine_similarity_tfidf(idx1, idx2, words_text_sets, all_tfs, idfs):
    text1= words_text_sets[idx1]
    text2= words_text_sets[idx2]
    tfs1=all_tfs[idx1]
    tfs2=all_tfs[idx2]

    common_words = text1 & text2
    if len(common_words)==0: return 0.0
    common_tfidfs = [tfs1[w]*tfs2[w]*idfs[w]*idfs[w] for w in common_words]

    #squared tfidfs
    tfidfs2_1=np.array([tfs1[w]*idfs[w] for w in text1])**2
    tfidfs2_2=np.array([tfs2[w]*idfs[w] for w in text2])**2

    return sum(common_tfidfs)/(np.sqrt(tfidfs2_1.sum())*np.sqrt(tfidfs2_2.sum()))

dst=cosine_similarity_tfidf(1, 3, words_text_sets, all_tfs, idfs)
print("The distance between stories 1 and 3 is %7.4f" % ( dst)) 

we can now compute all distances between all pairs of documents and find the most similar

In [ ]:
def text_similarities2(words_text_sets, all_tfs, idfs):
    N=len(words_text_sets)
    sims=[]
    for i in range(N-1):
        for j in range(i+1, N):
            sim = cosine_similarity_tfidf(i,j, words_text_sets, all_tfs, idfs)
            sims.append((sim, (i,j)))
    return sims

sims = text_similarities2(words_text_sets, all_tfs, idfs)

sims = sorted(sims, reverse=True)
for J, text_pair in sims[:5]: 
    print( "%s has similarity: %7.4f" % (text_pair, J))

This is all very nice, but what is the performance impact of this new similarity metric?

In [ ]:
%timeit sims = text_similarities2(words_text_sets, all_tfs, idfs)

#### Exercises: 
1. What are the common words between documents 6 and 16, and justify why they appear closer. 
2. analise and comment the results
    1. Are all words equally impoertant?
    2. What might improve the results?
    3. What problems would this entail?

In [ ]:
# !!!! Solution to exercise 1 !!!!!!


## Expanding this approach to a larger corpus

We are going to use a very small sample from [the Wikipedia from Nov 2020](https://www.kaggle.com/datasets/ltcmdrdata/plain-text-wikipedia-202011) that is made available in Kaggle. The full dataset is about 24 GB of data, of which we are only using 40 MB in JSON file for a total of about 14,500 documents. [the quality of the entries is very variable with occasional hypertext markings in several entries]

Python makes it easy to process these files by using the json standard library

In [ ]:
import json

data = json.load(open('file1.json', "rt", encoding="utf-8"))
  
print("Number of docs:", len(data))
print("Showing first 10 titles")
for d in data[:10]:
    print(d["id"],"--->", len(d["text"]) , "[", d["title"], "]")

First let's create a new corpus from this data and then make some text processing and identify all words and all unique words as before

**Note:** this should take slightly longer as 14,500 documents are significantly more difficult to process than 18 short fables

In [ ]:
corpus = [d["text"] for d in data]

words_texts     = get_words_from_corpus(corpus)
words_text_sets = [set(words) for words in words_texts]
all_words       = calc_all_words(words_text_sets)

print("N. of texts:", len(corpus))

As before let's compute the TF and IDFs for all documents and words in the corpus

In [ ]:
idfs = IDF(all_words, words_text_sets)
all_tfs=calc_tfs(words_texts)


### Time for a break

Before we continue let's examine what's ahead: We need to compute the distances of all documents to all documents and **this is a quadratic time operation $O(N^2)$**

The number of computations should be:

$$nc= \dfrac{N.(N-1)}{2}$$

so for 14,679 documents it should be

In [ ]:
N=len(corpus)
print(N*(N-1)/2)

Assuming we have sufficient memory to accomodate these results, how long will it take for processing it?

Let's start with 20 documents

In [ ]:
sims = text_similarities2(words_text_sets[:20], all_tfs, idfs)

sims = sorted(sims, reverse=True)
for J, text_pair in sims[:5]: 
    print( "%s has similarity: %7.4f" % (text_pair, J))


Let's check documents 6 and 13 as they look really similar

In [ ]:
print(corpus[6])
print()
print(corpus[13])

Also let's time this using python's common `time()` function

In [ ]:
from time import time
time1=time()
sims = text_similarities2(words_text_sets[:20], all_tfs, idfs)
time2=time()
print("the operation took %8.6f seconds" % (time2-time1))

#### Solved Exercises

1. increase the number of documents to search similarities from up to 500 documents
2. plot the number of documents vs execution time (use matplotlib!)
3. Comment your results
4. Estimate how much would it take for processing the full corpus (~14500 documents)

In [ ]:
ndocs=range(50, 501, 50)
times=[]
for n in ndocs:
    t1=time()
    sims = text_similarities2(words_text_sets[:n], all_tfs, idfs)
    t2=time()
    print("%3d --> %8.4f seconds" % (n, t2-t1)) 
    times.append(t2-t1)

In [ ]:
import matplotlib.pyplot as plt

times=np.array(times)

plt.plot(ndocs, times)
plt.title("NDocs vs Time")
plt.grid()
plt.show()

This could perhaps be better visualised if we transform the times into its square root, where its linear dimension should become apparent

In [ ]:

plt.plot(ndocs, np.sqrt(times))
plt.title("NDocs vs sqrt(Time)")
plt.grid()
plt.show()

In [ ]:
# !!!!!!!!!!!!! Solution of exercise 4!!!!!!!!!!!!!!!!
# compute slope
slope=(np.sqrt(times)[-1]-np.sqrt(times)[1])/(ndocs[-1]-ndocs[1])
print("To compute the entire corpus it would take %6.3f hours" % ((slope*len(corpus))**2/3600))


## Entering ray - Easy distributed processing for Python

[Ray](https://www.ray.io/) is a very powerful library for distributed computing that allows load distribution between different logical units on the same machine, using GPUs, and different servers. Setting up a ray cluster is very easy and the same exact procedures used for distributing heavy computing loads in a machine, are the same if you have a data cluster or working in a cloud computing environment

ray is not installed by default so it must be installed on the command line with `pip install ray` or, if you feel lucky, you may tray the line below


In [ ]:
#!pip install ray

Ray is alwys initialized with ray.init()

In [ ]:
import ray

ray.init(ignore_reinit_error=True)


as a very simple example on how ray works let's create a dummy function that just wastes time (like a normal function would do). This function just uses the sleep function for sitting idle and the time (in seconds) it takes is a parameter and returns a simple string

In [ ]:
import time

def dummy(sleep):
    time.sleep(sleep)
    return f"Sleep time {sleep}"

sleep_times = [1.5, 0.1, 0.2, 0.1, 0.5, 0.7, 0.5, 0.4, 1.5, 1.3, 0.7, 0.6, 0.3, 0.8]

t1=time.time()
results=[dummy(st) for st in sleep_times]
t2=time.time()

print("The time spent was:", t2-t1)
print("The time expected of sleep would be:", sum(sleep_times))
for res in results: print(res)

now we can parallelize this with ray by decorating the function as a remote function (with `@ray.remote`), so ray will make use of resources avaliable to assign each task to each logical unit. and if there are more tasks than processing units, ray will manage the process and as soon as a new resource becomes available, it will be assigned.

The results from a remote function may have not been completed, they are a `future`, which means a promise that the result will become ready soon. to actually get the result we have use the method `ray.get()`

So to parallelize our dummy problem we will rewrite our code like this:

In [ ]:
@ray.remote
def dummy(sleep):
    time.sleep(sleep)
    return f"Sleep time {sleep}"


t1=time.time()
res_futures=[dummy.remote(st) for st in sleep_times]
t2=time.time()
results = ray.get(res_futures) 
t3=time.time()
print("The time spent for dispatching the tasks:", t2-t1)
print("The time spent for getting the results:", t3-t2)
print("The total time spent:", t3-t1)
print("The order of the results is:")
for res in results: print(res)


## A local Alternative to Ray - Using Python's multiprocessing 

the `multiprocessing` library allows for fast local processing taking advantage of many cores a machine may have, whithout the overload of ray for handling  much more complex situations 

The Multiprocessing module works by creating a pool of processes that can be run asynchronously and we access these resources by mapping each data item to each process. As such we will use an actual `map` function, that takes two arguments: a function and a list containing the data to be processed

Here is a simple example with the `dummy` sleep function

In [ ]:
from time import sleep, time

def dummy(sleep_time):
    print("starting to sleep for:", sleep_time)
    sleep(sleep_time)
    return f"Sleep time {sleep_time}"

from multiprocessing.pool import ThreadPool as Pool

sleep_times = [1.5, 0.1, 0.2, 0.1, 0.5, 0.7, 0.5, 0.4, 1.5, 1.3, 0.7, 0.6, 0.3, 0.8]
t1=time()
with Pool(5) as p:
    res = p.map(dummy, sleep_times)
t2=time()
print("The time spent for finishing:", t2-t1)


Now to apply this resource to our problem we need to define our problem as a mapping problem, taking note that the map only accepts one argument and our similarity function actually requires 4 arguments (the row to process, the full set of words, the tfs and idfs for all words)

Two alternatives would be possible:
* Use global variables - which would eventually be solved by the multiprocessing environment creating local copies
* Redefine our list of arguments, where each element is tuple constituted of all the required data which is later unpacked by the distributed function

We will follow the second approach which can be trivially implemented by slightly rewriting the involved functions


In [ ]:

def cos_sim_mp(args):
    text1, text2, tfs1, tfs2, idx1, idx2, idfs = args
    common_words = text1 & text2
    if len(common_words)==0: return 0.0, (idx1, idx2)
    common_tfidfs = [tfs1[w]*tfs2[w]*idfs[w]*idfs[w] for w in common_words]

    #squared tfidfs
    tfidfs2_1=np.array([tfs1[w]*idfs[w] for w in text1])**2
    tfidfs2_2=np.array([tfs2[w]*idfs[w] for w in text2])**2

    return sum(common_tfidfs)/(np.sqrt(tfidfs2_1.sum())*np.sqrt(tfidfs2_2.sum())), (idx1, idx2)


def text_similarities_mp(words_text_sets, all_tfs, idfs):
    N = len(words_text_sets)
    #here we define the new arguments
    
    args=[(words_text_sets[i], 
           words_text_sets[j],
           all_tfs[i], 
           all_tfs[j], i,j, idfs) for i in range(N-1) for j in range(i+1, N)]
    with Pool(None) as p:
        sims=p.map(cos_sim_mp, args)
    
    return sims

In [ ]:

sims = text_similarities_mp(words_text_sets[:20], all_tfs, idfs)
sims = sorted(sims, reverse=True)
for J, text_pair in sims[:5]: 
    print( "%s has similarity: %7.4f" % (text_pair, J))


#### Exercises
1. Measure and plot the performance of this approach by testing 10, 20,...,100 documents and compare it to the other approach

In [ ]:
# HERE try it!


### Using Distributed processing with Ray for speeding up similarity calculations

**WARNING** The code below, for some Ray installations, may be **EXTREMELY SLOW!**  - It is outside the scope of this course to guarantee a common, fast and reliable Ray installation

We will rewrite slightly our cosine similarity so that it may be easier to process and integrate with our previous code. The only thing is that this function will ruturn not only the similarity but a tuple of the indexes of the two documents involved. This function is not going to be declared remote.

What will be declared remote is a new function that will encompass the inside loop of the `text_similarities`, so that each core will receive one document as its basis and process its similarities to all the others

In [ ]:


@ray.remote
def text_row_sims(i, words_text_sets, all_tfs, idfs):
    N=len(words_text_sets)
    sims=[]
    for j in range(i+1, N):
        res = cos_sim_ray(i, j, words_text_sets, all_tfs, idfs)
        sims.append(res)
    return sims



Finally the most important function, the base call that is going to distribute the load to all the workers. As before we will call the remote function for each base document

Yet one further aspect must be referred to. Sending the `word text sets`, the `idfs` and the `tfs` to each worker (that amay not even be on the same machine, might be a very time consuming process, as these resources will be common to all the distributed processing. As such ray allows the creation of copies of those resources in a common shared spaced that can be accessed thorgh special references. These references are then used instead of the actual parameters to the remote function, that is then able to use them without actually being continuously receiving them. The only thing that the remote function receives is a reference that will allow the access to this resource

In [ ]:
from functools import reduce

def text_similarities_ray(words_text_sets, all_tfs, idfs):
    N=len(words_text_sets)
    r_wts=ray.put(words_text_sets)
    r_tfs=ray.put(all_tfs)
    r_idfs=ray.put(idfs)
    res=[]
    for i in range(N-1):
        res_futures=text_row_sims.remote(i, r_wts, r_tfs, r_idfs)
        res.append(res_futures)
    sims=ray.get(res)
    return reduce(lambda a,b: a+b, sims)

Let's confirm that the results are the same as before

In [ ]:
sims=text_similarities_ray(words_text_sets[:20], all_tfs, idfs)
sims = sorted(sims, reverse=True)
for J, text_pair in sims[:5]: 
    print( "%s has similarity: %7.4f" % (text_pair, J))


#### Exercises [if you can get a reasonably fast Ray installation]

1. Measure the performance of this approach by testing 10, 20,...,100 documents
2. Can you explain the performance differences?
3. Plot the (ndocs, time) graph and comment the results
4. in what situations would this approach be clearly better than the single processing?

In [ ]:
# !!!!!! Solução problema 1 !!!!!!!!!!!!!!


In [ ]:
# !!!!!! Solução problema 3 !!!!!!!!!!!!!!
